# Sentiment140

On this noteboook, we try to predict the polarity of a twitt.

https://medium.com/@alyafey22/sentiment-classification-from-keras-to-the-browser-7eda0d87cdc6

In [1]:
# imports
import matplotlib.pyplot as plt
from loguru import logger
import tensorflow as tf
import pandas as pd 
from sklearn.model_selection import train_test_split

from pathlib import Path

import numpy as np
 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GRU, Embedding
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.datasets import mnist
import re


Using TensorFlow backend.


In [2]:
from pkg import constants
from pkg import main
from pkg import multi_partner_learning
from pkg import scenario
from pkg import utils
from pkg.dataset import Dataset

# Download and unzip data if needed

In [ ]:
!curl https://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip --output trainingandtestdata.zip
!unzip trainingandtestdata.zip

# Load it

In [3]:
df_train = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding = "raw_unicode_escape", header=None)
df_test = pd.read_csv("testdata.manual.2009.06.14.csv", encoding = "raw_unicode_escape",  header=None)



df_train.columns = ["polarity", "id", "date", "query", "user", "text"]
df_test.columns = ["polarity", "id", "date", "query", "user", "text"]

# We keep only a fraction of the whole dataset

df_train = df_train.sample(frac = 0.1)

x = df_train["text"]
y = df_train["polarity"]

# Preprocessing functions

In [4]:
def process(txt):
    out = re.sub(r'[^a-zA-Z0-9\s]', '', txt)
    out = out.split()
    out = [word.lower() for word in out]
    return out

def getMax(data):
    max_tokens = 0 
    for txt in data:
        if max_tokens < len(txt.split()):
            max_tokens = len(txt.split())
    return max_tokens


def tokenize(thresh = 5):
    count  = dict()
    idx = 1
    word_index = dict()
    for txt in x:
        words = process(txt)
        for word in words:
            if word in count.keys():
                count[word] += 1
            else:
                count[word]  = 1
    most_counts = [word for word in count.keys() if count[word]>=thresh]
    for word in most_counts:
        word_index[word] = idx
        idx+=1
    return word_index


def create_sequences(data):
    tokens = []
    for txt in data:
        words = process(txt)
        seq = [0] * max_tokens
        i = 0 
        for word in words:
            start = max_tokens-len(words)
            if word.lower() in word_index.keys():
                seq[i+start] = word_index[word]
            i+=1
        tokens.append(seq)        
    return np.array(tokens)

def preprocess_dataset_labels(y):
    #y = np_utils.to_categorical(y, num_classes)
    y = np.array([e/4 for e in y])
    return y

In [5]:
max_tokens = getMax(x)


num_words = None
word_index = tokenize()
num_words = len(word_index)

input_shape = max_tokens
num_classes = len(np.unique(y))


print('length of the dictionary ',len(word_index))
print('max token ', max_tokens) 
print('num classes', num_classes)

length of the dictionary  15086
max token  39
num classes 2


# Let's see how our preprocessing works

In [6]:
process("Th**is. is -a- #test ")

['this', 'is', 'a', 'test']

In [7]:
print(create_sequences(["Th**is. is -a- #test "]))

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   37   96  123 2424]]


# Create custom scenario from mandatory parametters

In [8]:
scenario_params = {
    'partners_count': 3,
    'amounts_per_partner': [0.2, 0.5, 0.3],
}

# Set values for optinal parametters

In [9]:
scenario_params['epoch_count'] = 10
scenario_params['minibatch_count'] = 3

#### Every other parametter will be set to its default value

In [10]:
experiment_path = Path(r"C:\GitHub\distributed-learning-contributivity\experiments\nlp")

In [11]:
current_scenario = scenario.Scenario(
        scenario_params,
        experiment_path
    )

2020-08-10 13:26:46.543 | DEBUG    | pkg.scenario:__init__:52 - Dataset selected: mnist
2020-08-10 13:26:46.544 | DEBUG    | pkg.scenario:__init__:87 - Computation use the full dataset for scenario #1
2020-08-10 13:26:46.619 | INFO     | pkg.scenario:__init__:279 - ### Description of data scenario configured:
2020-08-10 13:26:46.620 | INFO     | pkg.scenario:__init__:280 -    Number of partners defined: 3
2020-08-10 13:26:46.620 | INFO     | pkg.scenario:__init__:281 -    Data distribution scenario chosen: random
2020-08-10 13:26:46.621 | INFO     | pkg.scenario:__init__:282 -    Multi-partner learning approach: fedavg
2020-08-10 13:26:46.621 | INFO     | pkg.scenario:__init__:283 -    Weighting option: uniform
2020-08-10 13:26:46.622 | INFO     | pkg.scenario:__init__:284 -    Iterations parameters: 10 epochs > 3 mini-batches > 8 gradient updates per pass
2020-08-10 13:26:46.622 | INFO     | pkg.scenario:__init__:290 - ### Data loaded: mnist
2020-08-10 13:26:46.623 | INFO     | pkg.sc

# Create Data Set

In [12]:
(X_train, X_test) = train_test_split(create_sequences(x), shuffle = False)
(y_train, y_test) = train_test_split(y, shuffle = False)



In [13]:
# idk

X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [14]:
print(X_train[0])
print(y_train[0])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 2 3 4 5 6
 7 8]
0


# Create Model

In [15]:
def generate_new_model_for_dataset():
    model = Sequential()
    embedding_size = 8
    model.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='layer_embedding'))

    model.add(GRU(units=16, name = "gru_1",return_sequences=True))
    model.add(GRU(units=8, name = "gru_2" ,return_sequences=True))
    model.add(GRU(units=4, name= "gru_3"))
    model.add(Dense(1, activation='sigmoid',name="dense_1"))
    model.compile(loss='binary_crossentropy',
              optimizer="Adam",
              metrics=['accuracy'])
    return model

# Assignate dataset to scenario

In [16]:
current_scenario.dataset = Dataset(
    "my_dataset",
    X_train,
    X_test,
    y_train,
    y_test,
    input_shape,
    num_classes,
    preprocess_dataset_labels,
    generate_new_model_for_dataset
)

In [17]:
# Check Scenario name
print(current_scenario.dataset.name)

my_dataset


# Split train and validation sets

In [18]:
current_scenario.dataset.train_val_split()

# Legacy 

In [19]:
current_scenario.partners_list = []

# Run scenario

In [20]:
main.run_scenario(current_scenario)

2020-08-10 13:26:48.999 | INFO     | pkg.scenario:split_data:534 - ### Splitting data among partners:
2020-08-10 13:26:49.000 | INFO     | pkg.scenario:split_data:535 -    Simple split performed.
2020-08-10 13:26:49.000 | INFO     | pkg.scenario:split_data:536 -    Nb of samples split amongst partners: 77760
2020-08-10 13:26:49.000 | INFO     | pkg.scenario:split_data:538 -    Partner #0: 15552 samples with labels [0, 4]
2020-08-10 13:26:49.001 | INFO     | pkg.scenario:split_data:538 -    Partner #1: 38880 samples with labels [0, 4]
2020-08-10 13:26:49.002 | INFO     | pkg.scenario:split_data:538 -    Partner #2: 23328 samples with labels [0, 4]
2020-08-10 13:26:49.145 | DEBUG    | pkg.scenario:compute_batch_sizes:582 -    Compute batch sizes, partner #0: 648
2020-08-10 13:26:49.146 | DEBUG    | pkg.scenario:compute_batch_sizes:582 -    Compute batch sizes, partner #1: 1620
2020-08-10 13:26:49.147 | DEBUG    | pkg.scenario:compute_batch_sizes:582 -    Compute batch sizes, partner #2: 

2020-08-10 13:28:43.491 | DEBUG    | pkg.multi_partner_learning:log_collaborative_round_partner_result:513 - Epoch 01/09 > Minibatch 02/02 > Partner id #0 (0/2) > val_acc: 0.63
2020-08-10 13:28:48.700 | DEBUG    | pkg.multi_partner_learning:log_collaborative_round_partner_result:513 - Epoch 01/09 > Minibatch 02/02 > Partner id #1 (1/2) > val_acc: 0.64
2020-08-10 13:28:54.429 | DEBUG    | pkg.multi_partner_learning:log_collaborative_round_partner_result:513 - Epoch 01/09 > Minibatch 02/02 > Partner id #2 (2/2) > val_acc: 0.64
2020-08-10 13:28:54.433 | DEBUG    | pkg.multi_partner_learning:compute_collaborative_round_fedavg:303 - End of fedavg collaborative round.
2020-08-10 13:28:56.584 | INFO     | pkg.multi_partner_learning:compute_test_score:184 -    Model evaluation at the end of the epoch: ['0.683', '0.635']
2020-08-10 13:28:56.585 | DEBUG    | pkg.multi_partner_learning:compute_test_score:187 -       Checking if early stopping criteria are met:
2020-08-10 13:28:56.586 | DEBUG    |

2020-08-10 13:31:01.214 | DEBUG    | pkg.multi_partner_learning:compute_collaborative_round_fedavg:259 - Start new fedavg collaborative round ...
2020-08-10 13:31:01.215 | DEBUG    | pkg.multi_partner_learning:compute_collaborative_round_fedavg:271 - (fedavg) Minibatch n°0 of epoch n°4, init aggregated model for each partner with models from previous round
2020-08-10 13:31:09.917 | DEBUG    | pkg.multi_partner_learning:log_collaborative_round_partner_result:513 - Epoch 04/09 > Minibatch 00/02 > Partner id #0 (0/2) > val_acc: 0.68
2020-08-10 13:31:15.754 | DEBUG    | pkg.multi_partner_learning:log_collaborative_round_partner_result:513 - Epoch 04/09 > Minibatch 00/02 > Partner id #1 (1/2) > val_acc: 0.69
2020-08-10 13:31:22.560 | DEBUG    | pkg.multi_partner_learning:log_collaborative_round_partner_result:513 - Epoch 04/09 > Minibatch 00/02 > Partner id #2 (2/2) > val_acc: 0.68
2020-08-10 13:31:22.564 | DEBUG    | pkg.multi_partner_learning:compute_collaborative_round_fedavg:303 - End o

2020-08-10 13:33:27.214 | DEBUG    | pkg.multi_partner_learning:compute_collaborative_round_fedavg:271 - (fedavg) Minibatch n°1 of epoch n°6, init aggregated model for each partner with models from previous round
2020-08-10 13:33:36.535 | DEBUG    | pkg.multi_partner_learning:log_collaborative_round_partner_result:513 - Epoch 06/09 > Minibatch 01/02 > Partner id #0 (0/2) > val_acc: 0.74
2020-08-10 13:33:42.275 | DEBUG    | pkg.multi_partner_learning:log_collaborative_round_partner_result:513 - Epoch 06/09 > Minibatch 01/02 > Partner id #1 (1/2) > val_acc: 0.76
2020-08-10 13:33:48.197 | DEBUG    | pkg.multi_partner_learning:log_collaborative_round_partner_result:513 - Epoch 06/09 > Minibatch 01/02 > Partner id #2 (2/2) > val_acc: 0.75
2020-08-10 13:33:48.201 | DEBUG    | pkg.multi_partner_learning:compute_collaborative_round_fedavg:303 - End of fedavg collaborative round.
2020-08-10 13:33:48.203 | DEBUG    | pkg.multi_partner_learning:compute_collaborative_round_fedavg:259 - Start new f

2020-08-10 13:36:02.174 | DEBUG    | pkg.multi_partner_learning:log_collaborative_round_partner_result:513 - Epoch 08/09 > Minibatch 02/02 > Partner id #0 (0/2) > val_acc: 0.77
2020-08-10 13:36:07.849 | DEBUG    | pkg.multi_partner_learning:log_collaborative_round_partner_result:513 - Epoch 08/09 > Minibatch 02/02 > Partner id #1 (1/2) > val_acc: 0.77
2020-08-10 13:36:14.289 | DEBUG    | pkg.multi_partner_learning:log_collaborative_round_partner_result:513 - Epoch 08/09 > Minibatch 02/02 > Partner id #2 (2/2) > val_acc: 0.76
2020-08-10 13:36:14.293 | DEBUG    | pkg.multi_partner_learning:compute_collaborative_round_fedavg:303 - End of fedavg collaborative round.
2020-08-10 13:36:16.460 | INFO     | pkg.multi_partner_learning:compute_test_score:184 -    Model evaluation at the end of the epoch: ['0.507', '0.767']
2020-08-10 13:36:16.461 | DEBUG    | pkg.multi_partner_learning:compute_test_score:187 -       Checking if early stopping criteria are met:
2020-08-10 13:36:16.462 | DEBUG    |

0

# Results

In [21]:
df_results = current_scenario.to_dataframe()
df_results["random_state"] = 1
df_results["scenario_id"] = 1
print(df_results.columns)

Index(['aggregation_weighting', 'dataset_fraction_per_partner', 'dataset_name',
       'epoch_count', 'final_relative_nb_samples',
       'gradient_updates_per_pass_count', 'is_early_stopping',
       'learning_computation_time_sec', 'minibatch_count',
       'mpl_nb_epochs_done', 'mpl_test_score',
       'multi_partner_learning_approach', 'nb_samples_used', 'partners_count',
       'samples_split_description', 'scenario_name', 'short_scenario_name',
       'test_data_samples_count', 'train_data_samples_count', 'random_state',
       'scenario_id'],
      dtype='object')


In [22]:
print(df_results.mpl_test_score)

0    0.77085
Name: mpl_test_score, dtype: float64


In [23]:
print(current_scenario.mpl)

# Extract model 

In [24]:
model = current_scenario.mpl.get_model()

In [25]:
model.evaluate(X_test, preprocess_dataset_labels(y_test))

40000/40000 [==============================] - 25s 631us/step


[0.5026198272585869, 0.7708500027656555]

Seems nice as a federated learning result